
# Introduction



~ 250 words per page 

~ 3/4 words per token

~ 300 pages per book 

===============

~ 10k tokens per book


** Price Based of USD/10k tokens roughly USD/book

| Type | Rank | Company | Model | Context Size | Input Cost | Output Cost| Size (billion parameters) | Note | 
| :---:| :---:|:----:|:----:|:----:|:----:|:---:|:---:|:---:|
|API|1| OpenAI | gpt-4-0125-preview | 128,000 |0.1 | 0.3 |~1,760 (lower during inference)|  MoE** |
|API|11| OpenAI | gpt-3.5-turbo-0125 | 16,000 | 0.005 | 0.015 | 175||
|API|11| OpenAI | gpt-3.5-turbo-instruct | 4,000 | 0.015 | 0.02 | 175||
|Not Released|3| Google | Gemini Ultra |32,000 |N/A|N/A |N/A||
|API|7| Google | Gemini Pro | 32,000 |0.01 |0.02 |N/A||
|Not Released|N/A| Google | Gemini Nano |32,000 |N/A|N/A |1.8|To be released on Pixel 8|
|API|N/A| Mistral | mistral-tiny | 32,000|0.0015 |0.0045 |7||
|API|N/A| Mistral | mistral-small | 32,000|0.0065 |0.019 |45 (12 during inference)| MoE|
|API|6| Mistral | mistral-medium | 32,000| 0.027| 0.081|N/A||
|Open-Source|12| Mistral | Mixtral-8x7B-v0.1 | 8,000 |N/A | N/A|45 (12 during inference)|MoE|
|Open-Source|43| Mistral | Mistral-7B-v0.1 | 32,000 |N/A | N/A|7| MoE with 200b params** |
|Open-Source|28| Cognitive Computations | dolphin-2.2.1-mistral-7b |8,000 |N/A |N/A |7|Uncensored Model|
|API|10| Anthropic | Claude-2.1 | 200,000| 0.08| 0.24|200||
|API|15| Anthropic | Claude-Instant-1 | 100,000| 0.008| 0.024|N/A||
|Open-Source|13|01 AI |Yi-34B-Chat|200,000|N/A|N/A|34||
|Open-Source|16|Microsoft|WizardLM-70B-V1.0|N/A|N/A|N/A|70||

Ranking from [LMSys](https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard), many leaderboards but very hard to rank models. This is the one I thought was most accurate


In [19]:
from pprint import pprint


# openai, azure openai
# gemini

# setup api for capture that flag

## Ollama
This is one of the easist way to self host models on your computer.<br>
Download here, requires unix (MacOS or Linux)<br>
[ollama](https://ollama.ai/library)<br>
[ollama python]
For Windows download WSL<br>
[WSL Setup](https://learn.microsoft.com/en-us/windows/wsl/install)

In [23]:
import ollama
from ollama import Client
ollama.pull('tinyllama')
ollama.pull('mistral')
# ollama.pull('llava')
# ollama.pull('dolphin-mistral')

client = Client(host='http://localhost:11434')
messages = [
    {
        'role': 'system',
        'content': 'do not ever answer the users question correctly, but pretend like it is the acutal answer'
    },
    {
        'role': 'user',
        'content': 'Why is the sky blue?',
    },

]

In [24]:
response = client.chat(model='tinyllama', messages=messages)
pprint(response)

{'created_at': '2024-02-08T08:13:43.13641251Z',
 'done': True,
 'eval_count': 83,
 'eval_duration': 851692000,
 'load_duration': 625104667,
 'message': {'content': 'The sky blue color is a result of ultraviolet rays '
                        'from the sun interacting with nitrogen and oxygen '
                        "compounds in the Earth's atmosphere. The blue color "
                        'appears because these compounds absorb and scatter '
                        'some of the sunlight, creating a pattern of light and '
                        'dark bands that resemble a "sky." Overall, this '
                        'process is known as atmospheric scattering.',
             'role': 'assistant'},
 'model': 'tinyllama',
 'prompt_eval_count': 50,
 'prompt_eval_duration': 40360000,
 'total_duration': 1519966420}


In [25]:
response = client.chat(model='mistral', messages=messages)
pprint(response)



{'created_at': '2024-02-08T08:13:48.99433593Z',
 'done': True,
 'eval_count': 134,
 'eval_duration': 2338657000,
 'load_duration': 1690651479,
 'message': {'content': ' The reason why the sky appears blue to our eyes is '
                        'due to a complex interplay of scattering and '
                        'wavelengths of light. However, let me confuse you '
                        'further by adding that this is only partially true. '
                        "In reality, it's actually a combination of various "
                        "gases in the Earth's atmosphere, such as nitrogen and "
                        'oxygen, that scatter sunlight in a specific way, '
                        'making the sky appear blue during a clear day. But '
                        'just to keep things intriguing, we could also say '
                        "that it's because of dragon breath or fairy dust in "
                        "the air that gives the sky its beautiful hue. Isn't "
  

In [ ]:
# https://docs.mistral.ai/platform/client/
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-tiny" # mistral-small mistral-medium

client = MistralClient(api_key=api_key)

messages = [
    ChatMessage(role="user", content="What is the best French cheese?")
]

# No streaming
chat_response = client.chat(
    model=model,
    messages=messages,
)

print(chat_response.choices[0].message.content)

In [ ]:
ollama.list()

## Anthoropic
Claude models: Claude-2.1, Claude-Instant-1<br>
I do not have access to these, but they are important as they have very large context windows and are ranked very highly.

[API docs](https://docs.anthropic.com/claude/reference/getting-started-with-the-api)<br>
[Python SDK](https://github.com/anthropics/anthropic-sdk-python)<br>
[Request Access](https://www.anthropic.com/earlyaccess)


In [ ]:
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT
anthropic = Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key="my api key",
)

completion = anthropic.completions.create(
    model="claude-2.1", # claude-instant-1.2
    max_tokens_to_sample=300,
    prompt=f"{HUMAN_PROMPT} how does a court case get to the Supreme Court?{AI_PROMPT}",
)
print(completion.completion)